In [42]:
import requests
import pandas as pd
import time
import os

In [43]:
def create_tables(BASE_URL: str, headers: dict, measure_category_code: str, measure_code: str):
#     # CREATE MEASURE TABLE
#     measure_response = requests.get(f'{BASE_URL}/measure-categories/{measure_category_code}/measures', headers=headers).json()
#     measure_list = []
#     for i in measure_response['result']:
#         measure_list.append([i['measure_code'], i['measure_name']])
#     measures_df = pd.DataFrame(data=measure_list, columns=['measure_code', 'measure_name'])
#     measures_df.name = 'measures'
    # CREATE VALUES TABLE
    value_response = requests.get(f'{BASE_URL}/measures/{measure_code}/data-items', headers=headers).json()
    value_list = []
    for result in value_response['result']:
        value_list.append([result['reported_measure_code'], result['reporting_unit_summary']['reporting_unit_code'], result['value']])
    values_df = pd.DataFrame(data=value_list, columns=['reported_measure_code', 'reporting_unit_code', 'value'])
    values_df.name = 'values'
    # CREATE REPORTED MEASURES TABLE
    reported_measure_code_list = list(values_df['reported_measure_code'].unique())
    reported_measure_list = []
    for reported_measure_code in reported_measure_code_list:
        reported_measure_response = requests.get(f'{BASE_URL}/reported-measures/{reported_measure_code}', headers=headers).json()
        reported_measure_list.append([reported_measure_code, reported_measure_response['result']['reported_measure_name']])
    reported_measures_df = pd.DataFrame(data=reported_measure_list, columns=['reported_measure_code', 'reported_measure_name'])
    reported_measures_df.name = 'reported_measures'
    # CREATE REPORTING UNITS TABLE
    reporting_unit_response = requests.get(f'{BASE_URL}/reporting-units', headers=headers).json()   

    reporting_unit_list = []
    for result in reporting_unit_response['result']:
        # GET STATE
        mapped_reporting_units = result['mapped_reporting_units']
        state = None
        for mapped_reporting_unit in mapped_reporting_units:
            if mapped_reporting_unit['map_type']['mapped_reporting_unit_code'] == "STATE_MAPPING":
                state = mapped_reporting_unit['mapped_reporting_unit']['reporting_unit_code']
                break # Set state as the first reporting_unit_code in mapped_reporting_units

        # GET REPORTING UNIT INFO
        reporting_unit_list.append([result['reporting_unit_code'], result['reporting_unit_name'], result['reporting_unit_type']['reporting_unit_type_code'], result['reporting_unit_type']['reporting_unit_type_name'], state, result['closed'], result['private'], result['latitude'], result['longitude']])

    reporting_units_df = pd.DataFrame(data=reporting_unit_list, columns=['reporting_unit_code', 'reporting_unit_name', 'reporting_unit_type_code', 'reporting_unit_type_name', 'state', 'closed', 'private', 'latitude', 'longitude'])
    reporting_units_df.name = 'reporting_units'
    
#     # JOIN VALUES TABLE, REPORTED MEASURES TABLE AND REPORTING UNITS TABLE
#     df_join = pd.merge(values_df, reported_measures_df, on='reported_measure_code',how='inner')
#     df_join = pd.merge(df_join, reporting_units_df, on='reporting_unit_code',how='inner')
    
#     # SELECT COLUMNS
#     df_select = df_join[['reporting_unit_name', 'reported_measure_name', 'value']]
#     df_select.name = 'select'
    
    # CREATE LIST OF DATAFRAMES
    df_list = [values_df, reported_measures_df, reporting_units_df]
    return df_list

In [ ]:
BASE_URL ='https://myhospitalsapi.aihw.gov.au//api/v1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
create_tables(BASE_URL, headers, 'MYH-ED-TIME', 'MYH0036')[1].head()
# %timeit create_table('MYH-ED-TIME', MYH0036')

In [8]:
'''
CREATE MEASURE TABLE
EXAMPLE URL: https://myhospitalsapi.aihw.gov.au//api/v1/measure-categories/MYH-ED-TIME/measures
'''
measure_response = requests.get(f'{BASE_URL}/measure-categories/{measure_category_code}/measures', headers=headers).json()
measure_list = []
for i in measure_response['result']:
    measure_list.append([i['measure_code'], i['measure_name']])
measures_df = pd.DataFrame(data=measure_list, columns=['measure_code', 'measure_name'])
measures_df.head()

,measure_code,measure_name
0,MYH0005,Percentage of patients who depart the emergenc...
1,MYH0012,Number of patients presenting to the emergency...
2,MYH0013,Time until most patients (90%) departed the em...
3,MYH0036,Median time (50%) patients departed emergency ...


In [9]:
'''
CREATE VALUES TABLE
EXAMPLE URL: https://myhospitalsapi.aihw.gov.au//api/v1/measures/MYH0036/data-items
'''
measure_code = 'MYH0036'
value_response = requests.get(f'{BASE_URL}/measures/{measure_code}/data-items', headers=headers).json()
value_list = []
for result in value_response['result']:
    value_list.append([result['reported_measure_code'], result['reporting_unit_summary']['reporting_unit_code'], result['value']])
values_df = pd.DataFrame(data=value_list, columns=['reported_measure_code', 'reporting_unit_code', 'value'])
values_df.head()

,reported_measure_code,reporting_unit_code,value
0,MYH-RM0298,H0014,355.0
1,MYH-RM0299,H0014,170.0
2,MYH-RM0300,H0014,204.0
3,MYH-RM0298,H0014,338.0
4,MYH-RM0299,H0014,173.0


In [10]:
'''
CREATE REPORTED MEASURES TABLE
EXAMPLE URL: https://myhospitalsapi.aihw.gov.au//api/v1/reported-measures/MYH-RM0025
'''
reported_measure_code_list = list(values_df['reported_measure_code'].unique())

reported_measure_list = []
for reported_measure_code in reported_measure_code_list:
    reported_measure_response = requests.get(f'{BASE_URL}/reported-measures/{reported_measure_code}', headers=headers).json()
    reported_measure_list.append([reported_measure_code, reported_measure_response['result']['reported_measure_name']])
reported_measures_df = pd.DataFrame(data=reported_measure_list, columns=['reported_measure_code', 'reported_measure_name'])
reported_measures_df.head()

,reported_measure_code,reported_measure_name
0,MYH-RM0298,Subsequently admitted patients
1,MYH-RM0299,Not subsequently admitted patients
2,MYH-RM0300,All patients


In [11]:
'''
CREATE REPORTING UNITS TABLE
EXAMPLE URL: https://myhospitalsapi.aihw.gov.au/api/v1/reporting-units
'''
reporting_unit_response = requests.get(f'{BASE_URL}/reporting-units', headers=headers).json()   

reporting_unit_list = []
for result in reporting_unit_response['result']:
    # GET STATE
    mapped_reporting_units = result['mapped_reporting_units']
    state = None
    for mapped_reporting_unit in mapped_reporting_units:
        if mapped_reporting_unit['map_type']['mapped_reporting_unit_code'] == "STATE_MAPPING":
            state = mapped_reporting_unit['mapped_reporting_unit']['reporting_unit_code']
            break # Set state as the first reporting_unit_code in mapped_reporting_units
    
    # GET REPORTING UNIT INFO
    reporting_unit_list.append([result['reporting_unit_code'], result['reporting_unit_name'], result['reporting_unit_type']['reporting_unit_type_code'], result['reporting_unit_type']['reporting_unit_type_name'], state, result['closed'], result['private'], result['latitude'], result['longitude']])
    
reporting_units_df = pd.DataFrame(data=reporting_unit_list, columns=['reporting_unit_code', 'reporting_unit_name', 'reporting_unit_type_code', 'reporting_unit_type_name', 'state', 'closed', 'private', 'latitude', 'longitude',])
reporting_units_df.head()

,reporting_unit_code,reporting_unit_name,reporting_unit_type_code,reporting_unit_type_name,state,closed,private,latitude,longitude
0,H0012,State Forensic Mental Health Service,H,Hospital,WA,False,False,-31.960937,115.788431
1,H0013,Justice Health Services,H,Hospital,NSW,False,False,-33.969070,151.243206
2,H0014,The Children's Hospital at Westmead,H,Hospital,NSW,False,False,-33.801554,150.991759
3,H0015,Sydney Children's Hospital,H,Hospital,NSW,False,False,-33.917179,151.238334
4,H0016,Sacred Heart Health Service,H,Hospital,NSW,False,False,-33.880525,151.219237


In [12]:
'''
JOIN VALUES TABLE, REPORTED MEASURES TABLE AND REPORTING UNITS TABLE
'''
df_join = pd.merge(values_df, reported_measures_df, on='reported_measure_code',how='inner')
df_join = pd.merge(df_join, reporting_units_df, on='reporting_unit_code',how='inner')
df_join.head()

,reported_measure_code,reporting_unit_code,value,reported_measure_name,reporting_unit_name,reporting_unit_type_code,reporting_unit_type_name,state,closed,private,latitude,longitude
0,MYH-RM0298,H0014,355.0,Subsequently admitted patients,The Children's Hospital at Westmead,H,Hospital,NSW,False,False,-33.801554,150.991759
1,MYH-RM0298,H0014,338.0,Subsequently admitted patients,The Children's Hospital at Westmead,H,Hospital,NSW,False,False,-33.801554,150.991759
2,MYH-RM0298,H0014,314.0,Subsequently admitted patients,The Children's Hospital at Westmead,H,Hospital,NSW,False,False,-33.801554,150.991759
3,MYH-RM0298,H0014,256.0,Subsequently admitted patients,The Children's Hospital at Westmead,H,Hospital,NSW,False,False,-33.801554,150.991759
4,MYH-RM0298,H0014,263.0,Subsequently admitted patients,The Children's Hospital at Westmead,H,Hospital,NSW,False,False,-33.801554,150.991759


In [13]:
'''
SELECT COLUMNS
'''
df_select = df_join[['reporting_unit_name', 'reported_measure_name', 'value']]
df_select.head()

,reporting_unit_name,reported_measure_name,value
0,The Children's Hospital at Westmead,Subsequently admitted patients,355.0
1,The Children's Hospital at Westmead,Subsequently admitted patients,338.0
2,The Children's Hospital at Westmead,Subsequently admitted patients,314.0
3,The Children's Hospital at Westmead,Subsequently admitted patients,256.0
4,The Children's Hospital at Westmead,Subsequently admitted patients,263.0


In [44]:
# SPARK
import pyspark

credentials_location = '/home/kelvin/.gc/gc-key.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "/home/kelvin/spark/spark-3.3.1-bin-hadoop3/lib/gcs-connector-hadoop3-2.2.5.jar,/home/kelvin/spark/spark-3.3.1-bin-hadoop3/lib/spark-3.1-bigquery-0.28.0-preview.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location) \
    .set("spark.driver.extraClassPath", "/home/kelvin/spark/spark-3.3.1-bin-hadoop3/lib/spark-bigquery-with-dependencies_2.12-0.28.0.jar")

sc = SparkContext.getOrCreate(conf=conf)
print('scWebURL:', sc.uiWebUrl)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

spark.conf.set("temporaryGcsBucket","de-project-bucket")

scWebURL: http://de-proj.australia-southeast2-a.c.de-zoomcamp-project-377704.internal:4040


In [6]:
spark.stop()

In [46]:
# DECLARE VARIABLES
LOCAL_BASE_PATH = 'data'
measure_category_code = 'MYH-ED-TIME'
measure_name = 'MYH0036'
measure_path = f'{LOCAL_BASE_PATH}/{measure_name}'
BASE_URL ='https://myhospitalsapi.aihw.gov.au//api/v1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
bucket_name = "de-project-bucket"

In [47]:
# LOAD PANDAS DFs TO LOCAL DISK AS PARQUET
df_list = create_tables(BASE_URL, headers, measure_category_code, measure_name)
if not os.path.exists(measure_path):
    os.makedirs(measure_path)
for df in df_list:
    file_path = os.path.join(f'{LOCAL_BASE_PATH}/{measure_name}', f'{df.name}.parquet')
    df.to_parquet(file_path)

In [33]:
# LOAD PARQUET FILES TO GCS
import subprocess
subprocess.run(['gsutil', '-m', 'cp', '-r', measure_path, f'gs://{bucket_name}/{measure_path}'])

Copying file://data/MYH0036/reported_measures.parquet [Content-Type=application/octet-stream]...
Copying file://data/MYH0036/reporting_units.parquet [Content-Type=application/octet-stream]...
Copying file://data/MYH0036/values.parquet [Content-Type=application/octet-stream]...
/ [3/3 files][ 77.5 KiB/ 77.5 KiB] 100% Done                                    
Operation completed over 3 objects/77.5 KiB.                                     


CompletedProcess(args=['gsutil', '-m', 'cp', '-r', 'data/MYH0036', 'gs://de-project-bucket/data/MYH0036'], returncode=0)

In [49]:
# JOIN SPARK DFs AND WRITE TO BIGQUERY
file_list = []
for df in df_list:
    file_list.append(f'{df.name}.parquet')
print(file_list)

df_values = spark.read.parquet(f'gs://{bucket_name}/{measure_path}/values.parquet')
df_reported_measures = spark.read.parquet(f'gs://{bucket_name}/{measure_path}/reported_measures.parquet')
df_reporting_units = spark.read.parquet(f'gs://{bucket_name}/{measure_path}/reporting_units.parquet')

df_join = df_values.join(df_reported_measures, on=['reported_measure_code'], how='inner')
df_join = df_join.join(df_reporting_units, on=['reporting_unit_code'], how='inner')

output_table = f'{measure_name}.median_wait_time'
df_join.write.format('bigquery') \
    .option('table', output_table) \
    .mode('overwrite') \
    .save()

['values.parquet', 'reported_measures.parquet', 'reporting_units.parquet']


In [ ]:
'''
JOIN VALUES TABLE, REPORTED MEASURES TABLE AND REPORTING UNITS TABLE
'''
df_join = pd.merge(values_df, reported_measures_df, on='reported_measure_code',how='inner')
df_join = pd.merge(df_join, reporting_units_df, on='reporting_unit_code',how='inner')
df_join.head()

In [ ]:
'''TO DO
Create select tables
- convert value to hour-minute

Check other measures e.g. number of presentations

Create a udf (to do a fancy table with insights)

'''